<a href="https://colab.research.google.com/github/HiteshAI/Projects/blob/master/nepali_sentiment_classifier_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.text import *
import pandas as pd

In [23]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving nepali_sentiment.csv to nepali_sentiment.csv
User uploaded file "nepali_sentiment.csv" with length 131528 bytes


In [0]:
df = pd.read_csv('nepali_sentiment.csv')

In [28]:
df.head()

,SENTENCETEXT\tLABEL
0,"""अब गाह्रो छैन प्रदूषण नाप्न भारतीय पूर्व प्रध..."
1,"""लाठी लिएका हजारौँ संख्यामा रहेका थारुहरुले र्..."
2,"""आफूहरुले आफनो स्वाभिमानको लडाइ लडेको उनको भना..."
3,"""प्रदर्शनकारीले प्रहरी दमन बन्द गर, कैलाली कञ्..."
4,"""भारतको अघोषित नाकाबन्दीका कारण पेट्रोलियम पदा..."


In [29]:
df.keys()

Index(['SENTENCETEXT\tLABEL'], dtype='object')

In [0]:
path = Path('/content')
print(path)
data_lm = (TextList.from_csv(path, 'nepali_sentiment.csv', cols='SENTENCETEXT', delimiter='\t')
                   .split_by_rand_pct()
                   .label_for_lm()
                   .databunch())

In [33]:
data_lm.show_batch()

idx,text
0,"xxunk xxunk , xxunk , xxunk xxunk निकै xxunk xxunk xxunk छन् "" xxbos "" विद्यालय xxunk भएकाले xxunk समेत xxunk xxunk , xxunk xxunk विद्यालय xxunk xxunk xxunk xxunk xxunk xxunk "" xxbos "" प्रमुख xxunk दल नेपाली xxunk नेतृत्वमा राष्ट्रिय सहमतिको सरकार गठन xxunk सो xxunk xxunk नेताहरुको xxunk रहेको छ "" xxbos "" xxunk xxunk नेतृत्वमा सहमतिको सरकार गठन गर्ने विषयमा केन्द्रीय समिति xxunk भए पनि नेतृत्व"
1,"गर्ने प्रक्रिया सुरु xxunk "" xxbos "" xxunk xxunk xxunk xxunk xxunk "" xxbos "" xxunk xxunk नभए xxunk थप xxunk गर्ने xxunk तयारी रहेको जानकारी xxunk "" xxbos "" xxunk xxunk राजनीतिक xxunk उनले प्रमुख xxunk दल xxunk एमालेले xxunk विरोधी आन्दोलन गरिरहेको xxunk xxunk "" xxbos "" उनले एमाले र xxunk xxunk xxunk संविधान xxunk xxunk xxunk xxunk xxunk xxunk हुने बताए "" xxbos "" तर सो xxunk"
2,"छ "" xxbos "" xxunk नेपाल प्रहरी र xxunk xxunk xxunk "" xxbos "" xxunk xxunk xxunk , व्यवस्थित र xxunk लागि भन्दै xxunk जिल्ला xxunk xxunk गठन गरिएको छ "" xxbos "" प्रतियोगिता फागुन १३ सम्म हुने xxunk जनाएको छ "" xxbos "" xxunk लागि xxunk xxunk पाँच हजार रुपैयाँ सहभागिता xxunk xxunk "" xxbos "" १० वटा xxunk सहभागी xxunk xxunk बनेको xxunk xxunk xxunk र xxunk प्राप्त"
3,"xxunk अर्को रुपमा xxunk "" xxbos "" xxunk xxunk xxunk xxunk xxunk xxunk पहिलो गोल गरेका थिए "" xxbos "" उनले xxunk xxunk xxunk xxunk xxunk xxunk xxunk गर्दै xxunk xxunk xxunk xxunk xxunk गोल गरेका थिए "" xxbos "" xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk गरे "" xxbos "" अझ , उनले xxunk लागि दुई वटा xxunk पनि xxunk थिए "" xxbos "" xxunk"
4,"पनि त्यो xxunk xxunk भएको थियो "" xxbos "" xxunk xxunk xxunk xxunk xxunk xxunk लागेको थियो "" xxbos "" xxunk छ xxunk xxunk xxunk लागि xxunk xxunk गरेको xxunk "" xxbos "" तर पनि कुनै xxunk xxunk xxunk xxunk भनेर xxunk पनि हुन xxunk "" xxbos "" xxunk लिएर xxunk भन्दा xxunk xxunk xxunk अनि xxunk मात्र xxunk xxunk "" xxbos "" xxunk xxunk सदस्य xxunk १ हजार रुपैयाँ"


In [34]:
learn = language_model_learner(data_lm, AWD_LSTM)
learn.fit_one_cycle(2, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,5.911543,4.703639,0.422991,00:17
1,5.736596,4.319719,0.461161,00:16


In [0]:
learn.save('mini_train_lm')
learn.save_encoder('mini_train_encoder')

In [36]:
learn.show_results()

text,target,pred
"xxbos "" xxunk xxunk xxunk xxunk रहेका xxunk xxunk xxunk गरी xxunk xxunk xxunk xxunk xxunk गरेका थिए "" xxbos",""" xxunk xxunk मा xxunk xxunk xxunk xxunk प्रतिशत xxunk xxunk xxunk पुगेको देखिन्छ , xxunk छ "" xxbos ""","xxmaj xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk , xxmaj"
"xxunk xxunk प्रतिशत xxunk xxunk xxunk पुगेको देखिन्छ , xxunk छ "" xxbos "" सरकारले xxunk xxunk xxunk घोषणा गरी","xxunk , xxunk xxunk , xxunk xxunk xxunk र xxunk xxunk xxunk xxunk xxunk वातावरण xxunk "" xxbos "" xxunk","xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk , xxmaj xxmaj"
"xxunk xxunk र xxunk xxunk xxunk xxunk xxunk वातावरण xxunk "" xxbos "" xxunk xxunk फेरि xxunk xxunk xxunk यसले","xxunk xxunk थियो "" xxbos "" xxunk सवारी xxunk ट्राफिक प्रहरीले xxunk xxunk xxunk व्यवस्थापन xxunk यो xxunk ट्राफिक xxunk","xxunk xxunk xxunk xxunk , xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk"
"xxunk सवारी xxunk ट्राफिक प्रहरीले xxunk xxunk xxunk व्यवस्थापन xxunk यो xxunk ट्राफिक xxunk xxunk पनि xxunk गरिएको हुन्छ ""","xxbos "" उनले xxunk यो कुरा xxunk गरेर xxunk xxunk xxunk "" xxbos "" कार्यक्रमको xxunk xxunk xxunk xxunk अध्यक्ष",", xxmaj xxmaj xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxbos xxmaj xxunk xxunk xxunk xxunk xxunk"
"xxunk गरेर xxunk xxunk xxunk "" xxbos "" कार्यक्रमको xxunk xxunk xxunk xxunk अध्यक्ष xxunk xxunk गरेका थिए "" xxbos",""" त्यसको xxunk काम xxunk हुन्छ र xxunk अधिकार xxunk xxunk xxunk अर्को सहमतिको सरकार xxunk xxunk xxunk "" xxbos","xxmaj xxmaj xxunk xxunk xxunk xxunk "" xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk xxunk"


In [0]:
data_class = (TextList.from_csv(path, 'nepali_sentiment.csv', cols='SENTENCETEXT', delimiter='\t', vocab=data_lm.vocab)
                   .split_by_rand_pct(0.1)
                   .label_from_df(cols='LABEL')
                   .databunch(bs=4))

In [42]:
data_class.show_batch()

text,target
"xxbos "" xxunk नेपाल xxunk xxunk xxunk , xxunk , xxunk , xxunk , xxunk , xxunk , xxunk , xxunk , xxunk र xxunk xxunk xxunk xxunk काम गर्दै आएको छ """,1
"xxbos "" xxunk xxunk , सामाजिक xxunk xxunk , आर्थिक समाजिक xxunk xxunk अवस्था , र सामाजिक xxunk xxunk अवस्था र xxunk xxunk गर्न समाजको xxunk xxunk उभिएर xxunk xxunk भने देखिन्छ """,0
"xxbos "" गोली xxunk xxunk xxunk गाविस २ का xxunk xxunk , xxunk गाविस ४ का xxunk xxunk , xxunk xxunk xxunk xxunk ज्यान गएको छ """,-1
"xxbos "" xxunk , xxunk र xxunk ती ट्राफिक xxunk xxunk xxunk शक्ति xxunk , xxunk xxunk र xxunk कुनै न कुनै xxunk देखिएको थियो """,-1


In [44]:
learn = text_classifier_learner(data_class, AWD_LSTM)
learn.load_encoder('mini_train_encoder')
learn.fit_one_cycle(2, slice(1e-3,1e-2))
learn.save('mini_train_clas')

epoch,train_loss,valid_loss,accuracy,time
0,1.185531,1.214188,0.266667,00:24
1,1.150573,1.116895,0.311111,00:24


In [45]:
learn.show_results()

text,target,prediction
"xxbos "" उनका अनुसार , xxunk गरेको xxunk एक वर्ष xxunk समय भएकाले xxunk xxunk डाक्टर xxunk काम xxunk , xxunk xxunk xxunk xxunk xxunk हुन्छ """,-1,0
"xxbos "" अध्यक्ष xxunk अनुसार xxunk xxunk xxunk xxunk प्रतियोगिता , xxunk , xxunk प्रतियोगिता , कृषि xxunk , xxunk xxunk xxunk कार्यक्रम हुने भएको छ """,1,0
"xxbos "" xxunk xxunk आफ्नो xxunk xxunk र xxunk xxunk xxunk xxunk - xxunk xxunk कालो कालो xxunk xxunk , यो पनि त्यस्तै हो भनिन्छ """,-1,1
"xxbos "" xxunk xxunk xxunk xxunk वर्ष मात्र सार्वजनिक गरेको xxunk xxunk xxunk xxunk प्रतिशत xxunk विवाह xxunk xxunk हुने गरेको सार्वजनिक गरेको थियो """,-1,1
